# Ejercicio 1: Ajuste de Hiperparámetros con TensorBoard

## Objetivo
Desarrollar un modelo de red neuronal para clasificación binaria ajustando hiperparámetros y visualizar el proceso de entrenamiento usando TensorBoard.

## Descripción
1. Utiliza el conjunto de datos sobre detección de spam proporcionado (https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)
2. Implementa una red neuronal con al menos 2 capas ocultas
3. Realiza un proceso de búsqueda de hiperparámetros probando:
   - Diferentes tasas de aprendizaje: [0.001, 0.01, 0.1]
   - Diferentes arquitecturas: [32-16, 64-32, 128-64] neuronas
   - Diferentes funciones de activación: ['relu', 'tanh']
4. Utiliza TensorBoard para visualizar:
   - Curvas de pérdida y precisión
   - Distribución de pesos por capa
   - Métricas de evaluación







In [6]:
from google.colab import drive
import pandas as pd

In [10]:
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Colab Notebooks/06_tensorboard/data/SMSSpamCollection.txt'

# Leer el archivo (si es CSV)
df = pd.read_csv(file_path, sep='\t', header=None, names=["label", "message"])

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Implementa una red neuronal con al menos 2 capas ocultas

# Convertir etiquetas a valores numéricos (spam=1, ham=0)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Convertir mensajes de texto en características numéricas (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['message']).toarray()
y = df['label'].values

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Salida binaria (spam o ham)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo: {accuracy:.4f}')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8619 - loss: 0.4271 - val_accuracy: 0.9874 - val_loss: 0.0638
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9900 - loss: 0.0402 - val_accuracy: 0.9883 - val_loss: 0.0479
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9972 - loss: 0.0079 - val_accuracy: 0.9892 - val_loss: 0.0478
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9993 - loss: 0.0033 - val_accuracy: 0.9892 - val_loss: 0.0529
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 0.9892 - val_loss: 0.0558
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 5.3934e-04 - val_accuracy: 0.9892 - val_loss: 0.0556
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 0.9892 - val_loss: 0.0596
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9994 - loss: 0.0026 - val_accura

In [12]:
import tensorflow as tf
from tensorflow import keras
from itertools import product
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Realiza un proceso de búsqueda de hiperparámetros probando:
# Diferentes tasas de aprendizaje: [0.001, 0.01, 0.1]
# Diferentes arquitecturas: [32-16, 64-32, 128-64] neuronas
# Diferentes funciones de activación: ['relu', 'tanh']

# Definir los hiperparámetros a probar
learning_rates = [0.001, 0.01, 0.1]
architectures = [(32, 16), (64, 32), (128, 64)]
activations = ['relu', 'tanh']

best_accuracy = 0
best_params = {}

# Probar todas las combinaciones de hiperparámetros
for lr, (neurons1, neurons2), activation in product(learning_rates, architectures, activations):
    print(f"Probando configuración: LR={lr}, Neuronas=({neurons1}-{neurons2}), Activación={activation}")

    # Crear el modelo
    model = keras.Sequential([
        keras.layers.Dense(neurons1, activation=activation, input_shape=(X_train.shape[1],)),
        keras.layers.Dense(neurons2, activation=activation),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compilar el modelo con la tasa de aprendizaje específica
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(X_test, y_test))

    # Evaluar el modelo
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Precisión obtenida: {accuracy:.4f}")

    # Guardar la mejor configuración
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'learning_rate': lr, 'neurons': (neurons1, neurons2), 'activation': activation}

# Imprimir la mejor configuración
print("\nMejor configuración encontrada:")
print(best_params)
print(f"Mejor precisión en el conjunto de prueba: {best_accuracy:.4f}")


Probando configuración: LR=0.001, Neuronas=(32-16), Activación=relu
Precisión obtenida: 0.9901
Probando configuración: LR=0.001, Neuronas=(32-16), Activación=tanh
Precisión obtenida: 0.9919
Probando configuración: LR=0.001, Neuronas=(64-32), Activación=relu
Precisión obtenida: 0.9883
Probando configuración: LR=0.001, Neuronas=(64-32), Activación=tanh
Precisión obtenida: 0.9892
Probando configuración: LR=0.001, Neuronas=(128-64), Activación=relu
Precisión obtenida: 0.9910
Probando configuración: LR=0.001, Neuronas=(128-64), Activación=tanh
Precisión obtenida: 0.9901
Probando configuración: LR=0.01, Neuronas=(32-16), Activación=relu
Precisión obtenida: 0.9910
Probando configuración: LR=0.01, Neuronas=(32-16), Activación=tanh
Precisión obtenida: 0.9839
Probando configuración: LR=0.01, Neuronas=(64-32), Activación=relu
Precisión obtenida: 0.9892
Probando configuración: LR=0.01, Neuronas=(64-32), Activación=tanh
Precisión obtenida: 0.9865
Probando configuración: LR=0.01, Neuronas=(128-64), 

In [13]:
import tensorflow as tf
from tensorflow import keras
from itertools import product
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import datetime  # Para nombrar los logs según la fecha

# Utiliza TensorBoard para visualizar:
# Curvas de pérdida y precisión
# Distribución de pesos por capa
# Métricas de evaluación


# Directorio para guardar logs de TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Callback de TensorBoard
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Probar todas las combinaciones de hiperparámetros
for lr, (neurons1, neurons2), activation in product(learning_rates, architectures, activations):
    print(f"Probando configuración: LR={lr}, Neuronas=({neurons1}-{neurons2}), Activación={activation}")

    # Crear el modelo
    model = keras.Sequential([
        keras.layers.Dense(neurons1, activation=activation, input_shape=(X_train.shape[1],)),
        keras.layers.Dense(neurons2, activation=activation),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compilar el modelo con la tasa de aprendizaje específica
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo con TensorBoard activado
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0,
              validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

    # Evaluar el modelo
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Precisión obtenida: {accuracy:.4f}")

    # Guardar la mejor configuración
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'learning_rate': lr, 'neurons': (neurons1, neurons2), 'activation': activation}

# Imprimir la mejor configuración
print("\nMejor configuración encontrada:")
print(best_params)
print(f"Mejor precisión en el conjunto de prueba: {best_accuracy:.4f}")

# Guardar modelo final con mejor configuración
model.save("best_model.h5")

print("Entrenamiento finalizado. Usa TensorBoard para visualizar los resultados.")


Probando configuración: LR=0.001, Neuronas=(32-16), Activación=relu
Precisión obtenida: 0.9901
Probando configuración: LR=0.001, Neuronas=(32-16), Activación=tanh
Precisión obtenida: 0.9910
Probando configuración: LR=0.001, Neuronas=(64-32), Activación=relu
Precisión obtenida: 0.9901
Probando configuración: LR=0.001, Neuronas=(64-32), Activación=tanh


KeyboardInterrupt: 

---



# Ejercicio 2: Transfer Learning y Fine-tuning con Monitoreo

## Objetivo
Aplicar transfer learning a un modelo pre-entrenado y realizar fine-tuning monitoreando el proceso con TensorBoard.

## Descripción
1. Utiliza un modelo pre-entrenado (ej: ResNet50) para la clasificación de imágenes
2. Implementa:
   - Congelamiento de capas base
   - Adición de nuevas capas para fine-tuning
   - Callbacks personalizados para TensorBoard
3. Monitorea durante el entrenamiento:
   - Gradientes por capa
   - Mapas de activación
   - Learning rate adaptativo
4. Compara el rendimiento:
   - Modelo base vs fine-tuned
   - Diferentes estrategias de congelamiento
   - Impacto del data augmentation